In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import sqlite3

plt.style.use('default')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

print("✅ Librerías importadas exitosamente!")

✅ Librerías importadas exitosamente!


In [5]:
df_original = pd.read_csv(r'/Users/Ayala/Documents/9A/Carsi/Integradora/Generacio_n_estimada_de_residuos_peligrosos_por_tipo_de_residuo_2017-2023.csv',
encoding='latin1')

print("✅ Archivo CSV cargado exitosamente!")
print(df_original.info())
df_original.head()

✅ Archivo CSV cargado exitosamente!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6375 entries, 0 to 6374
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID Entidad Federaativa  6375 non-null   int64  
 1   Entidad federativa      6375 non-null   object 
 2   Municipio               6375 non-null   object 
 3   Año                     6375 non-null   int64  
 4   Subsector               6335 non-null   object 
 5   Categoría de generador  6375 non-null   object 
 6   Categoría del residuo   6366 non-null   object 
 7   Tipo de residuo         6345 non-null   object 
 8   Generación estimada     6375 non-null   float64
dtypes: float64(1), int64(2), object(6)
memory usage: 448.4+ KB
None


,ID Entidad Federaativa,Entidad federativa,Municipio,Año,Subsector,Categoría de generador,Categoría del residuo,Tipo de residuo,Generación estimada
0,1,Aguascalientes,Sin Dato,2017,Petrolero,Microgenerador,"Gasolina, diesel y naftas gastados o sucios pr...",Pequeño Generador,1.22
1,1,Aguascalientes,Sin Dato,2017,Petrolero,Microgenerador,"Gasolina, diesel y naftas gastados o sucios pr...","Mezclas de combustibles sucios, mezclas de hid...",0.04
2,1,Aguascalientes,Sin Dato,2017,Petrolero,Microgenerador,Liquidos de proceso,"Agua contaminada con hidrocarburos, combustibl...",0.09
3,1,Aguascalientes,Sin Dato,2017,Petrolero,Microgenerador,Lodos aceitosos,"Con contaminacion de hidrocarburos, combustibl...",1.08
4,1,Aguascalientes,Sin Dato,2017,Petrolero,Microgenerador,Residuos solidos,"Estopa, trapo, plasticos, carton, madera, broc...",3.12


# Plan de Ejecución del Proceso ETL

Este documento detalla el plan de trabajo para el diseño e implementación de un proceso ETL (Extracción, Transformación, Carga) básico utilizando Python (principalmente Pandas) sobre nuestro conjunto de datos seleccionado de datos.gob.mx.

---

## 1. Integrantes del Equipo

* [Nombre del Integrante 1]
* [Nombre del Integrante 2]
* [Nombre del Integrante 3]
* [Nombre del Integrante N]

---

## 2. Dataset Seleccionado

* **Nombre del Dataset:** [Insertar el nombre del dataset aquí, ej., "Defunciones por COVID-19"]
* **Enlace al Dataset en datos.gob.mx:** [Insertar el enlace directo al dataset aquí, ej., `https://datos.gob.mx/busca/dataset/defunciones-por-covid-19`]

---

## 3. Objetivo de Análisis Principal

Nuestros análisis principales con este dataset para el proyecto integrador incluyen:

* [Detallar aquí 1-3 objetivos de análisis clave. Ej: "Analizar la evolución de los casos/defunciones a lo largo del tiempo.", "Identificar patrones geográficos o demográficos relevantes.", "Evaluar el impacto de ciertas variables en el fenómeno estudiado."]
* [Añadir otro objetivo si aplica]
* [Añadir otro objetivo si aplica]

Estas decisiones guiarán las transformaciones necesarias para preparar los datos.

---

## 4. Diseño Detallado del Proceso ETL

### 4.1. Plan de Extracción (E)

* **Archivo a Leer:** [Nombre del archivo, ej., `defunciones_covid.csv`]
* **Formato del Archivo:** [CSV, JSON, Excel, etc.]
* **Parámetros de Lectura (ej. para CSV):**
    * **Separador:** [`,` , `;` , `\t`, etc.]
    * **Encoding:** [Generalmente `'utf-8'` o `'latin1'`. Especificar si es necesario.]
    * **Encabezados:** [Sí/No, si no, especificar `header=None` y manejar nombres de columnas después.]
* **Selección Inicial de Columnas:**
    * [Indicar si se cargan todas las columnas o un subconjunto específico inicialmente. Ej: "Se cargarán todas las columnas y se realizará la selección en la fase de Transformación."]

### 4.2. Plan de Transformación (T)

Esta es la fase más crítica. Aquí detallamos las acciones que se tomarán para limpiar y estructurar los datos, columna por columna si es necesario, basándonos en nuestra exploración previa y los objetivos de análisis.

* **4.2.1. Renombrar/Estandarizar Columnas:**
    * **Lista de Columnas a Renombrar:**
        * `[Nombre Columna Original 1]` -> `[Nuevo Nombre Columna 1]` (Justificación: [Breve justificación])
        * `[Nombre Columna Original 2]` -> `[Nuevo Nombre Columna 2]` (Justificación: [Breve justificación])
        * [Añadir más si es necesario]

* **4.2.2. Selección de Columnas (Eliminación de No Necesarias):**
    * **Columnas Finales Necesarias:** [Listar todas las columnas que SÍ serán parte del DataFrame final y por qué son importantes para el análisis.]
    * **Columnas a Eliminar:** [Listar las columnas que se eliminarán y la razón (ej. no relevantes para el análisis, demasiados nulos).]

* **4.2.3. Corrección de Tipos de Datos:**
    * **Columna `[Nombre Columna 1]`:** Convertir a tipo `[Tipo Deseado, ej. datetime, int, float, category]`. (Justificación: [Breve justificación, ej. "Para realizar análisis temporales."])
    * **Columna `[Nombre Columna 2]`:** Convertir a tipo `[Tipo Deseado]`. (Justificación: [Breve justificación])
    * [Añadir más si es necesario]
    * *Consideraciones:* Usar `errors='coerce'` si hay valores que no se pueden convertir y se desean convertir a `NaT` (para fechas) o `NaN` (para numéricos).

* **4.2.4. Limpieza de Texto (para columnas categóricas/texto):**
    * **Columna `[Nombre Columna Categórica 1]`:**
        * Acciones: [Eliminar espacios extra (`.strip()`), convertir a minúsculas (`.lower()`), estandarizar valores (ej. "MEXICO" a "México").]
        * (Justificación: [Mejorar consistencia y permitir agrupaciones correctas.])
    * **Columna `[Nombre Columna Categórica 2]`:**
        * Acciones: [Detallar acciones específicas.]
    * [Añadir más si es necesario]

* **4.2.5. Manejo de Valores Faltantes (NaNs):**
    * **Columna `[Nombre Columna 1]`:**
        * Estrategia: [Ej. "Eliminar filas donde esta columna es NaN." o "Imputar con la media/mediana/moda." o "Rellenar con un valor constante como 'Desconocido'."]
        * Justificación: [Explicar por qué esta estrategia es adecuada para esta columna y el análisis.]
    * **Columna `[Nombre Columna 2]`:**
        * Estrategia: [Detallar estrategia específica.]
        * Justificación: [Justificar.]
    * [Añadir más para cada columna con NaNs relevantes]

* **4.2.6. Manejo de Duplicados:**
    * **Identificación:** [¿Se buscarán filas duplicadas completas o basadas en un subconjunto de columnas (ej. `['id_registro', 'fecha']`)?]
    * **Tratamiento:** [¿Eliminar la primera/última ocurrencia (`.drop_duplicates()`)?]
    * Justificación: [Ej. "Asegurar la unicidad de los registros clave."]

* **4.2.7. Validación/Corrección de Errores (Valores Atípicos/Imposibles):**
    * **Columna `[Nombre Columna Numérica/Fecha]`:**
        * Rango Lógico: [Ej. "Edad entre 0 y 120", "Fecha no posterior a la actual."]
        * Acción: [Ej. "Eliminar filas con valores fuera de rango", "Reemplazar por NaN y luego imputar", "Corregir si es un error de formato simple."]
        * Justificación: [Ej. "Evitar sesgos en promedios o análisis de tendencias."]
    * [Añadir más si es necesario]

* **4.2.8. (Opcional) Feature Engineering Básico:**
    * **Nueva Columna `[Nombre Nueva Columna 1]`:**
        * Origen: [Ej. "Extraer el mes y año de la columna `fecha_registro`."]
        * Cálculo: [Ej. `df['mes_registro'] = df['fecha_registro'].dt.month`]
        * Justificación: [Ej. "Permitir análisis de tendencias mensuales o anuales."]
    * **Nueva Columna `[Nombre Nueva Columna 2]`:**
        * Origen: [Ej. "Calcular una ratio de [variable_A] sobre [variable_B]."]
        * Cálculo: [Detallar fórmula.]
        * Justificación: [Justificar.]
    * [Añadir más si es necesario]

* **4.2.9. Estructura Final del DataFrame(s):**
    * Descripción: [Describir cómo se verá el DataFrame final. ¿Qué columnas tendrá? ¿Será uno solo o se dividirá en tablas conceptuales (ej. tabla de hechos y dimensiones si aplica)?]
    * Ejemplo de la estructura esperada:
        ```
        DataFrame Final:
        - Columna A (Tipo): Descripción
        - Columna B (Tipo): Descripción
        - ...
        ```

### 4.3. Plan de Carga (L)

* **Destino Simulado:** [Seleccionar una opción y justificar.]
    * **Opción 1 (Recomendada): Base de Datos SQLite**
        * Nombre del archivo de la DB: `[nombre_de_su_base_de_datos.db]`
        * Nombre de la tabla: `[nombre_de_su_tabla]`
        * Parámetros `to_sql`: `if_exists='replace'`, `index=False` (para no guardar el índice de Pandas como columna).
        * Justificación: [Ej. "Simula una carga a una base de datos relacional para futuros análisis SQL."]
    * **Opción 2: Archivos Limpios (CSV o JSON)**
        * Nombre del archivo CSV: `[nombre_dataset_limpio.csv]`
        * Nombre del archivo JSON: `[nombre_dataset_limpio.json]`
        * Parámetros `to_csv`/`to_json`: `index=False` (para no guardar el índice de Pandas como columna).
        * Justificación: [Ej. "Fácilmente accesible para herramientas de análisis posteriores."]

---

## 5. Implementación del ETL en Python (Sección para el Código)

En esta sección del notebook, se desarrollará el código Python. Cada paso del plan anterior debe tener su correspondiente bloque de código, acompañado de comentarios explicativos y verificaciones intermedias.

### 5.1. Importación de Librerías

```python
import pandas as pd
import sqlite3 # Solo si se elige SQLite como destino
# Importar cualquier otra librería necesaria (ej. numpy, datetime)